# Table of Contents
 <p>

In [1]:
from planet4 import region_data, io
import pvl
from hirise_tools import products, downloads, labels
import gdal

In [2]:
regions = ['Inca', 'Ithaca', 'Giza', 'Manhattan2']

In [3]:
seasons = ['season2', 'season3']

In [4]:
def get_fraction_of_black_pixels(savepath):
    ds = gdal.Open(str(savepath))
    data = ds.ReadAsArray()
    fractions = []
    for band in data:
        nonzeros = band.nonzero()[0]
        fractions.append((band.size - nonzeros.size)/band.size)
    return np.array(fractions).mean()

In [5]:
def read_metadata(obsid):
    prodid = products.PRODUCT_ID(obsid)
    prodid.kind = 'COLOR'
    labelpath = downloads.labels_root() / prodid.label_fname
    label = labels.HiRISE_Label(labelpath)
    savedir = downloads.hirise_dropbox() / 'browse'
    savepath = savedir / prodid.browse_path.name
    if not savepath.exists():
        ht.download_product(prodid.browse_path, savedir)
    black_fraction = get_fraction_of_black_pixels(savepath)
    all_area = label.line_samples*label.lines * label.map_scale**2
    real_area = (1-black_fraction)*all_area
    d = dict(obsid=obsid, path=labelpath, binning=label.binning,
             l_s=label.l_s, line_samples=label.line_samples,
             lines=label.lines, map_scale=label.map_scale, invalids=black_fraction,
             real_area=real_area)
    # self calculated north azimuths
    folder = io.get_ground_projection_root()
    path = folder / obsid / f"{obsid}_campt_out.csv"
    df = pd.read_csv(path)
    d['north_azimuth'] = df.NorthAzimuth.median()
    return d

In [6]:
read_metadata('ESP_021684_0985')

{'binning': [-9998,
  -9998,
  -9998,
  -9998,
  1,
  1,
  -9998,
  -9998,
  -9998,
  -9998,
  2,
  2,
  2,
  2],
 'invalids': 0.71131464528267252,
 'l_s': 253.373295,
 'line_samples': 69451,
 'lines': 15410,
 'map_scale': 0.25,
 'north_azimuth': 107.47709754428001,
 'obsid': 'ESP_021684_0985',
 'path': PosixPath('/Users/klay6683/Dropbox/data/hirise/labels/ESP_021684_0985_COLOR.LBL'),
 'real_area': 19310161.753186915}

In [7]:
for region in regions:
    rea = getattr(region_data, region)
    bucket = []
    if region.endswith('2'):
        region = region[:-1]
    print("Region:", region)
    for season in seasons:
        sea = getattr(rea, season)
        metadata = []
        for img in sea:
            metadata.append(read_metadata(img))
        df = pd.DataFrame(metadata)
        df['season'] = season
        bucket.append(df)
    
    name = "{}_metadata.csv".format(region.lower())
    fname = Path('/Users/klay6683/Dropbox/p4_4Chase') / region.lower() / name
    pd.concat(bucket, ignore_index=True).to_csv(str(fname), index=False)
    print("Created", fname)

Region: Inca
Created /Users/klay6683/Dropbox/p4_4Chase/inca/inca_metadata.csv
Region: Ithaca
Created /Users/klay6683/Dropbox/p4_4Chase/ithaca/ithaca_metadata.csv
Region: Giza
Created /Users/klay6683/Dropbox/p4_4Chase/giza/giza_metadata.csv
Region: Manhattan
Created /Users/klay6683/Dropbox/p4_4Chase/manhattan/manhattan_metadata.csv
